# 2020.03.28 (Sat)

## Base model: spaCy

In [1]:
#!prodigy train ner cord_19_abstracts_match models/en_core_web_lg_no_ner --output models/2020_03_28_match/en_rf_web_lg

✔ Loaded model 'models/en_core_web_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    2260.23       3.822      5.714      4.580                                 
 2     304.20      96.203     72.381     82.609                                 
 3      35.94      93.069     89.524     91.262                                 
 4       7.71      91.262     89.524     90.385                                 
 5       6.89      94.000     89.524     91.707                                 
 6       6.26      94.949     89.524     92.157                                 
 7       4.15      94.000     89.524     91.707                                 
 8      10.58      92.157   

## Base model: scispaCy

In [2]:
#!prodigy train ner cord_19_abstracts_match models/en_core_sci_lg_no_ner --output models/2020_03_28_match/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 86 total examples
Using 43 train / 43 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1864.33       3.333      2.941      3.125                                 
 2     303.79      93.902     75.490     83.696                                 
 3      22.39      92.784     88.235     90.452                                 
 4      17.47      94.792     89.216     91.919                                 
 5       5.36      93.814     89.216     91.457                                 
 6       6.78      92.857     89.216     91.000                                 
 7       6.09      93.814     89.216     91.457                                 
 8       4.87      94.792   

# 2020.03.29 (Sun)

## Test the model trained on Sat

In [1]:
import spacy
import warnings
from pprint import pprint

def test_model(model_path):
    warnings.filterwarnings("ignore",category=UserWarning)
    
    nlp = spacy.load(model_path)
    texts = [
    "Known risk factors for the disease are: older age, male gender, diabetes and leukemia. Female patiens and children are less susceptible.",
    "Diabetes is a known risk factor.",
    "Leukemia is a risk factor, too."
    ]

    for text in texts:
        doc = nlp(text)

        spacy.displacy.render(doc, style='ent', jupyter=True)
        pprint([(ent.text, ent.label_) for ent in doc.ents])

In [2]:
test_model("models/2020_03_28_match/en_rf_web_lg")

[('age', 'RISK_FACTOR'), ('male', 'RISK_FACTOR'), ('diabetes', 'RISK_FACTOR')]


[('Diabetes', 'RISK_FACTOR')]


[]


Only the factors that appeared during the training are detected. _Leukemia_ is **not highlighed** and that means that the NER model did not learn to recognize risk factors based on the sentence structure.

## Train new models after the `teach` session

In [1]:
#!prodigy train ner cord_19_abstracts_teach models/en_core_web_lg_no_ner --output models/2020_03_29_teach/en_rf_web_lg

✔ Loaded model 'models/en_core_web_lg_no_ner'
Created and merged data for 393 total examples
Using 197 train / 196 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     333.36      89.669     92.735     91.176                                 
 2      64.66      92.982     90.598     91.775                                 
 3      53.47      94.492     95.299     94.894                                 
 4      45.26      94.492     95.299     94.894                                 
 5      29.42      94.958     96.581     95.763                                 
 6      30.44      96.170     96.581     96.375                                 
 7      23.82      95.745     96.154     95.949                                 
 8       5.78      95.319

In [2]:
#!prodigy train ner cord_19_abstracts_teach models/en_core_sci_lg_no_ner --output models/2020_03_29_teach/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 393 total examples
Using 197 train / 196 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     289.28      90.991     91.818     91.403                                 
 2      54.12      92.237     91.818     92.027                                 
 3      84.39      91.593     94.091     92.825                                 
 4      46.45      93.213     93.636     93.424                                 
 5      19.43      95.000     95.000     95.000                                 
 6      15.25      93.665     94.091     93.878                                 
 7       4.07      93.363     95.909     94.619                                 
 8      11.23      93.722

In [3]:
test_model("models/2020_03_29_teach/en_rf_web_lg")

[('age', 'RISK_FACTOR'),
 ('male gender', 'RISK_FACTOR'),
 ('diabetes', 'RISK_FACTOR')]


[('Diabetes', 'RISK_FACTOR')]


[]


Better model scores, but _leukemia_ still **not detected**.

# 2020.03.30 (Mon)

## Starting the annotations from scratch (`strict`)

Terms like _male_ or _age_ will only be marked, when they are mentioned in the context of COVID-19 risk factors, i.e. not in articles about animals or children.

In [5]:
!prodigy ner.manual cord_19_abstracts_strict models/en_core_sci_lg_no_ner data/raw/cord_19_abstracts_filtered.jsonl \
  --label RISK_FACTOR --patterns patterns/RF_list_2020.03.17.20037572.jsonl

Using 1 label(s): RISK_FACTOR
Added dataset cord_19_abstracts_strict to database SQLite.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 41 annotations to database SQLite
Dataset: cord_19_abstracts_strict
Session ID: 2020-03-30_20-24-26



In [7]:
#!prodigy train ner cord_19_abstracts_strict models/en_core_sci_lg_no_ner --output models/2020_03_30_strict/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 41 total examples
Using 21 train / 20 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1737.49       0.000      0.000      0.000                                 
 2      25.99       0.000      0.000      0.000                                 
 3     115.63       0.000      0.000      0.000                                 
 4      24.86       0.000      0.000      0.000                                 
 5      22.36       0.000      0.000      0.000                                 
 6      18.33       0.000      0.000      0.000                                 
 7      20.52       0.000      0.000      0.000                                 
 8      28.45       0.000   

## Starting the annotations from scratch (`all RF`)

Highlight all risk factors - for any disease or species - but only is sentences that clear say it is a risk factor.

In [ ]:
!prodigy ner.manual cord_19_abstracts_all_rf models/en_core_sci_lg_no_ner data/raw/cord_19_abstracts_filtered.jsonl \
  --label RISK_FACTOR --patterns patterns/RF_highlight_factor_phrases.jsonl

In [6]:
!prodigy train ner cord_19_abstracts_all_rf models/en_core_sci_lg_no_ner --output models/2020_03_30_all_RF/en_rf_sci_lg

✔ Loaded model 'models/en_core_sci_lg_no_ner'
Created and merged data for 58 total examples
Using 29 train / 29 eval (split 50%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1960.54       0.000      0.000      0.000                                 
 2      95.38       0.000      0.000      0.000                                 
 3     161.78     100.000      2.128      4.167                                 
 4      88.78      40.000      4.255      7.692                                 
 5      97.87      33.333      4.255      7.547                                 
 6      87.81      37.500      6.383     10.909                                 
 7     341.72      25.000      2.128      3.922                                 
 8     169.64      40.000   

In [7]:
!prodigy ner.teach cord_19_abstracts_all_rf models/2020_03_30_all_RF/en_rf_sci_lg data/raw/cord_19_abstracts_filtered.jsonl \
  --label RISK_FACTOR

Using 1 label(s): RISK_FACTOR

✨  Starting the web server at http://localhost:8081 ...
Open the app in your browser and start annotating!

^C

✔ Saved 7 annotations to database SQLite
Dataset: cord_19_abstracts_all_rf
Session ID: 2020-03-30_22-01-26



# 2020.04.06 (Mon)

## Starting the annotations from scratch (`RF sentences`)

Following the official guide:
- https://www.youtube.com/watch?v=59BKHO_xBPA
- https://github.com/explosion/projects/tree/master/ner-food-ingredients#data-creation-and-training-workflow

### Label data manually

In [2]:
!prodigy ner.manual cord_19_rf_sentences blank:en data/raw/cord_19_rf_sentences.jsonl \
    --label RISK_FACTOR

Using 1 label(s): RISK_FACTOR

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 102 annotations to database SQLite
Dataset: cord_19_rf_sentences
Session ID: 2020-04-06_22-08-56



### Pretrain different models

#### `en_vectors_web_lg`

In [5]:
!prodigy train ner cord_19_rf_sentences en_vectors_web_lg --eval-split 0.2

✔ Loaded model 'en_vectors_web_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     389.18       0.000      0.000      0.000                                 
 2     191.03      25.000      4.000      6.897                                 
 3     265.45      35.294     24.000     28.571                                 
 4     246.57      28.571     24.000     26.087                                 
 5     239.21      22.727     20.000     21.277                                 
 6     215.80      42.857     24.000     30.769                                 
 7     258.40      42.105     32.000     36.364                                 
 8     237.19      50.000     32.000   

#### `en_core_web_lg`

In [6]:
!prodigy train ner cord_19_rf_sentences en_core_web_lg --eval-split 0.2

✔ Loaded model 'en_core_web_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1421.81       0.000      0.000      0.000                                 
 2    1245.16       0.000      0.000      0.000                                 
 3    1262.77       0.000      0.000      0.000                                 
 4    1249.04       0.000      0.000      0.000                                 
 5    1262.46       0.000      0.000      0.000                                 
 6    1205.26       0.000      0.000      0.000                                 
 7    1137.92       0.000      0.000      0.000                                 
 8    1131.84       0.000      0.000      

⁉️ Why is `en_core_web_lg` producing no results at all?

#### `en_core_sci_lg`

In [14]:
!prodigy train ner cord_19_rf_sentences en_core_sci_lg --eval-split 0.2 --output models/2020_04_06_rf_sentences

✔ Loaded model 'en_core_sci_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     559.17       0.000      0.000      0.000                                 
 2     550.33      33.333      8.000     12.903                                 
 3     538.43      50.000     24.000     32.432                                 
 4     457.89      52.632     40.000     45.455                                 
 5     452.06      50.000     36.000     41.860                                 
 6     415.01      60.000     48.000     53.333                                 
 7     489.45      47.826     44.000     45.833                                 
 8     461.13      45.000     36.000     4

#### `en_vectors_web_lg` + tok2vec

In [8]:
!prodigy train ner cord_19_rf_sentences en_vectors_web_lg --eval-split 0.2 \
    --init-tok2vec models/tok2vec_rf_vectors_model436.bin

✔ Loaded model 'en_vectors_web_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
✔ Initializing with tok2vec weights
models/tok2vec_rf_vectors_model436.bin
embed_rows: 2000 | require_vectors: True | cnn_maxout_pieces: 3 |
token_vector_width: 96 | conv_depth: 4 | nr_feature_tokens: 3 |
pretrained_vectors: en_vectors_web_lg.vectors
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     339.88       0.000      0.000      0.000                                 
 2     199.85      50.000     16.000     24.242                                 
 3     128.26      50.000     28.000     35.897                                 
 4      76.43      42.857     24.000     30.769                                 
 5      63.28      60.000     3

#### `en_core_web_lg` + tok2vec

In [9]:
!prodigy train ner cord_19_rf_sentences en_core_web_lg --eval-split 0.2 \
    --init-tok2vec models/tok2vec_rf_vectors_model436.bin

✔ Loaded model 'en_core_web_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
✔ Initializing with tok2vec weights
models/tok2vec_rf_vectors_model436.bin

ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    2055.18       0.000      0.000      0.000                                 
 2    1808.48       0.000      0.000      0.000                                 
 3    1624.76       0.000      0.000      0.000                                 
 4    1443.31       0.000      0.000      0.000                                 
 5    1652.93       0.000      0.000      0.000                                 
 6    1470.22       0.000      0.000      0.000                                 
 7    1345.14       0.000      0.000      0.000

#### `en_core_sci_lg` + tok2vec

In [13]:
!prodigy train ner cord_19_rf_sentences en_core_sci_lg --eval-split 0.2 \
    --init-tok2vec models/tok2vec_rf_sci_model328.bin

✔ Loaded model 'en_core_sci_lg'
Created and merged data for 99 total examples
Using 80 train / 19 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 10
✔ Initializing with tok2vec weights
models/tok2vec_rf_sci_model328.bin

ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1     536.68       0.000      0.000      0.000                                 
 2     631.61       0.000      0.000      0.000                                 
 3     539.99      40.000     24.000     30.000                                 
 4     505.27      47.619     40.000     43.478                                 
 5     508.05      45.000     36.000     40.000                                 
 6     461.69      52.941     36.000     42.857                                 
 7     473.25      45.000     36.000     40.000    

### Label more data by correcting the model's predictions

The best model so far will be used: `Best F-Score   53.333`

In [15]:
!prodigy ner.correct cord_19_rf_sentences_correct models/2020_04_06_rf_sentences data/raw/cord_19_rf_sentences.jsonl \
    --label RISK_FACTOR --exclude cord_19_rf_sentences

Using 1 label(s): RISK_FACTOR
Added dataset cord_19_rf_sentences_correct to database SQLite.

✨  Starting the web server at http://localhost:8080 ...
Open the app in your browser and start annotating!

^C

✔ Saved 220 annotations to database SQLite
Dataset: cord_19_rf_sentences_correct
Session ID: 2020-04-06_23-58-48



### Train the final model

In [16]:
!prodigy train ner cord_19_rf_sentences,cord_19_rf_sentences_correct en_core_sci_lg --eval-split 0.2 --n-iter 20

✔ Loaded model 'en_core_sci_lg'
Created and merged data for 308 total examples
Using 247 train / 61 eval (split 20%)
Component: ner | Batch size: compounding | Dropout: 0.2 | Iterations: 20
ℹ Baseline accuracy: 0.000

=========================== ✨  Training the model ===========================

#    Loss       Precision   Recall     F-Score 
--   --------   ---------   --------   --------
 1    1554.25      61.538     37.500     46.602                                 
 2    1346.09      64.407     59.375     61.789                                 
 3    1407.15      66.667     50.000     57.143                                 
 4    1411.73      72.727     50.000     59.259                                 
 5    1412.81      73.913     53.125     61.818                                 
 6    1319.96      73.913     53.125     61.818                                 
 7    1378.05      72.917     54.688     62.500                                 
 8    1208.99      72.727     62.500    